In [ ]:
from bw2data import projects, databases, Database, methods
import bw2regional as bwr
from bw2calc import LCA
from bw2regional.export import as_ia_spatial_scale
from bw2analyzer import ContributionAnalysis

In [ ]:
projects.set_current("EXIOBASE regionalized case study")

In [ ]:
[x for x in methods if x[0] == "LC-IMPACT"]

In [ ]:
sorted(bwr.extension_tables)

In [ ]:
palm = next(ds for ds in Database("Oil seeds") if ds['name'] == 'Palm oil refinery [NBD palm oil]')
sunflower = next(ds for ds in Database("Oil seeds") if ds['name'] == 'Sunflower oil refinery [Refined sunflower oil]')
rapeseed = next(ds for ds in Database("Oil seeds") if ds['name'] == 'Rapeseed oil refinery [Refined rapeseed oil]')
peanut = next(ds for ds in Database("Oil seeds") if ds['name'] == 'Peanut oil refinery [Refined peanut oil]')
soybean = next(ds for ds in Database("Oil seeds") if ds['name'] == 'Soybean oil refinery [CSBO processing]')

In [ ]:
for obj in (rapeseed, sunflower, peanut, palm, soybean):
    print(obj.key)

In [ ]:
lca = LCA(
    {palm: 1},
    ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
)
lca.lci()
lca.lcia()
lca.score

In [ ]:
lca = bwr.TwoSpatialScalesLCA(
    {palm: 1},
    ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
)
lca.lci()
lca.lcia()
lca.score

In [ ]:
lca = bwr.ExtensionTablesLCA(
    {palm: 1},
    ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
    xtable='xt-oilpalm-world-topo-ecoregions',
)
lca.lci()
lca.lcia()
lca.score

In [ ]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(lca)[:10]:
        print([act['name'], act['unit'], act['location'], score / lca.score])